# Nested Logit Model: Compute Elasticities (Direct, Cross, Arc)

In [1]:
import pandas as pd
import numpy as np
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.optimization as opt
import biogeme.results as res
from biogeme.expressions import Beta, DefineVariable, Derive
import seaborn as sns
import matplotlib.pyplot as plt

**Import Optima data**

In [2]:
pandas = pd.read_csv("../../Data/8-DiscreteChoiceModels/optima.dat",sep='\t')
database = db.Database ("data/optima", pandas)

**Use collumn names as variables**

In [3]:
globals().update(database.variables)

**Exclude some unwanted entries**

In [4]:
exclude = (Choice == -1.)
database.remove(exclude)

**Define some dummy variables**

In [5]:
male = (Gender == 1)
female = (Gender == 2)
unreportedGender = (Gender == -1)

fulltime = (OccupStat == 1)
notfulltime = (OccupStat != 1)

**Rescale some data**

In [6]:
TimePT_scaled = TimePT / 200
TimeCar_scaled = TimeCar / 200
MarginalCostPT_scaled = MarginalCostPT / 10 
CostCarCHF_scaled = CostCarCHF / 10
distance_km_scaled = distance_km / 5

**Compute normalizing weights for each alternative**

In [7]:
sumWeight = database.data['Weight'].sum()
normalized_Weight = Weight * len(database.data['Weight']) / sumWeight

**Create parameters to be estimated**

In [8]:
ASC_CAR              = Beta('ASC_CAR',0,None,None,0)
ASC_PT               = Beta('ASC_PT',0,None,None,1)
ASC_SM               = Beta('ASC_SM',0,None,None,0)
BETA_TIME_FULLTIME   = Beta('BETA_TIME_FULLTIME',0,None,None,0)
BETA_TIME_OTHER      = Beta('BETA_TIME_OTHER',0,None,None,0)
BETA_DIST_MALE       = Beta('BETA_DIST_MALE',0,None,None,0)
BETA_DIST_FEMALE     = Beta('BETA_DIST_FEMALE',0,None,None,0)
BETA_DIST_UNREPORTED = Beta('BETA_DIST_UNREPORTED',0,None,None,0)
BETA_COST            = Beta('BETA_COST',0,None,None,0)

**Define the utility functions**

\begin{align}
V_{PT} & = \beta_{PT} + \beta_{time_{fulltime}} X_{time_{PT}} X_{fulltime} + \beta_{time_{other}} X_{time_{PT}} X_{not\_fulltime} + \beta_{cost} X_{cost_{PT}} \\
V_{car} & = \beta_{car} + \beta_{time_{fulltime}} X_{time_{car}} X_{fulltime} + \beta_{time_{other}} X_{time_{car}} X_{not\_fulltime} + \beta_{cost} X_{cost_{car}} \\
V_{SM} & = \beta_{SM} + \beta_{male} X_{distance} X_{male} + \beta_{female} X_{distance} X_{female} + \beta_{unreported} X_{distance} X_{unreported}
\end{align}

In [9]:
V_PT = ASC_PT + BETA_TIME_FULLTIME * TimePT_scaled * fulltime + \
       BETA_TIME_OTHER * TimePT_scaled * notfulltime + \
       BETA_COST * MarginalCostPT_scaled
V_CAR = ASC_CAR + \
        BETA_TIME_FULLTIME * TimeCar_scaled * fulltime + \
        BETA_TIME_OTHER * TimeCar_scaled * notfulltime + \
        BETA_COST * CostCarCHF_scaled
V_SM = ASC_SM + \
       BETA_DIST_MALE * distance_km_scaled * male + \
       BETA_DIST_FEMALE * distance_km_scaled * female + \
       BETA_DIST_UNREPORTED * distance_km_scaled * unreportedGender

**Associate utility functions with alternatives and associate availability of alternatives**

In this example all alternatives are available for each individual

In [10]:
V = {0: V_PT,
     1: V_CAR,
     2: V_SM}

av = {0: 1,
      1: 1,
      2: 1}

**Define the nests**

1. Define the nests paramenters
2. List alternatives in nests

In [11]:
MU_NO_CAR = Beta('MU_NO_CAR', 1.,1.,None,0)

CAR_NEST = 1., [1]
NO_CAR_NEST = MU_NO_CAR, [0, 2]

nests = CAR_NEST, NO_CAR_NEST

**Define the model**

In [12]:
prob_pt  = models.nested(V,av,nests,0)
prob_car = models.nested(V,av,nests,1)
prob_sm  = models.nested(V,av,nests,2)

## Aggregate Direct Elasticities

**Calculate the direct elasticities**

In [13]:
direct_elas_pt_time  = Derive(prob_pt,'TimePT') * TimePT / prob_pt 
direct_elas_pt_cost  = Derive(prob_pt,'MarginalCostPT') * MarginalCostPT / prob_pt 
direct_elas_car_time = Derive(prob_car,'TimeCar') * TimeCar / prob_car 
direct_elas_car_cost = Derive(prob_car,'CostCarCHF') * CostCarCHF / prob_car 
direct_elas_sm_dist  = Derive(prob_sm,'distance_km') * distance_km / prob_sm

**Define what we want to simulate**

In [14]:
simulate = {'weight': normalized_Weight,
            'Prob. car': prob_car,
            'Prob. public transportation': prob_pt,
            'Prob. slow modes':prob_sm,
            'direct_elas_pt_time':direct_elas_pt_time,
            'direct_elas_pt_cost':direct_elas_pt_cost,
            'direct_elas_car_time':direct_elas_car_time,
            'direct_elas_car_cost':direct_elas_car_cost,
            'direct_elas_sm_dist':direct_elas_sm_dist}

**Define the Biogeme object**

In [15]:
biogeme = bio.BIOGEME(database, simulate)
biogeme.modelName = "optima_nested_logit_elasticities_direct"

**Retrieve the names of the variables we want to use. Then retrieve the results from the model that we estimated earlier**

In [16]:
betas = biogeme.freeBetaNames

results = res.bioResults(pickleFile='optima_nested_logit.pickle')
betaValues = results.getBetaValues ()

**Perform the simulation**

In [17]:
simulatedValues = biogeme.simulate(betaValues)

**Calculate the elasticities**

In [18]:
simulatedValues['Weighted prob. car'] = simulatedValues['weight'] * simulatedValues['Prob. car']
simulatedValues['Weighted prob. PT']  = simulatedValues['weight'] * simulatedValues['Prob. public transportation']
simulatedValues['Weighted prob. SM']  = simulatedValues['weight'] * simulatedValues['Prob. slow modes']

denominator_car = simulatedValues['Weighted prob. car'].sum()
denominator_pt = simulatedValues['Weighted prob. PT'].sum()
denominator_sm = simulatedValues['Weighted prob. SM'].sum()

direct_elas_term_car_time = (simulatedValues['Weighted prob. car'] * simulatedValues['direct_elas_car_time'] / denominator_car).sum()
direct_elas_term_car_cost = (simulatedValues['Weighted prob. car'] * simulatedValues['direct_elas_car_cost'] / denominator_car).sum()
direct_elas_term_pt_time = (simulatedValues['Weighted prob. PT'] * simulatedValues['direct_elas_pt_time'] / denominator_pt).sum()
direct_elas_term_pt_cost = (simulatedValues['Weighted prob. PT'] * simulatedValues['direct_elas_pt_cost'] / denominator_pt).sum()
direct_elas_term_sm_dist = (simulatedValues['Weighted prob. SM'] * simulatedValues['direct_elas_sm_dist'] / denominator_sm).sum()

**Print the resulting aggregate direct elasticities**

In [19]:
print(f"Aggregate direct elasticity of car wrt time: {direct_elas_term_car_time:.3g}")
print(f"Aggregate direct elasticity of car wrt cost: {direct_elas_term_car_cost:.3g}")
print(f"Aggregate direct elasticity of PT wrt time: {direct_elas_term_pt_time:.3g}")
print(f"Aggregate direct elasticity of PT wrt cost: {direct_elas_term_pt_cost:.3g}")
print(f"Aggregate direct elasticity of SM wrt distance: {direct_elas_term_sm_dist:.3g}")

Aggregate direct elasticity of car wrt time: -0.044
Aggregate direct elasticity of car wrt cost: -0.0906
Aggregate direct elasticity of PT wrt time: -0.274
Aggregate direct elasticity of PT wrt cost: -0.32
Aggregate direct elasticity of SM wrt distance: -1.09


## Aggregate Cross Elasticities

**Calculate the cross elasticities**

In [20]:
cross_elas_car_time = Derive(prob_car,'TimePT') * TimePT / prob_car 
cross_elas_car_cost = Derive(prob_car,'MarginalCostPT') * MarginalCostPT / prob_car 
cross_elas_pt_time = Derive(prob_pt,'TimeCar') * TimeCar / prob_pt 
cross_elas_pt_cost = Derive(prob_pt,'CostCarCHF') * CostCarCHF / prob_pt 

**Define what we want to simulate**

In [21]:
simulate = {'weight': normalized_Weight,
            'Prob. car': prob_car,
            'Prob. public transportation': prob_pt,
            'Prob. slow modes':prob_sm,
            'cross_elas_pt_time':cross_elas_pt_time,
            'cross_elas_pt_cost':cross_elas_pt_cost,
            'cross_elas_car_time':cross_elas_car_time,
            'cross_elas_car_cost':cross_elas_car_cost}

**Define the Biogeme object**

In [22]:
biogeme = bio.BIOGEME(database, simulate)
biogeme.modelName = "optima_nested_logit_elasticities_cross"

**Retrieve the names of the variables we want to use. Then retrieve the results from the model that we estimated earlier**

In [23]:
betas = biogeme.freeBetaNames

results = res.bioResults(pickleFile='optima_nested_logit.pickle')
betaValues = results.getBetaValues ()

**Perform the simulation**

In [24]:
simulatedValues = biogeme.simulate(betaValues)

**Calculate the elasticities**

In [25]:
simulatedValues['Weighted prob. car'] = simulatedValues['weight'] * simulatedValues['Prob. car']
simulatedValues['Weighted prob. PT'] = simulatedValues['weight'] * simulatedValues['Prob. public transportation']

denominator_car = simulatedValues['Weighted prob. car'].sum()
denominator_pt = simulatedValues['Weighted prob. PT'].sum()

cross_elas_term_car_time = (simulatedValues['Weighted prob. car'] * simulatedValues['cross_elas_car_time'] / denominator_car).sum()
cross_elas_term_car_cost = (simulatedValues['Weighted prob. car'] * simulatedValues['cross_elas_car_cost'] / denominator_car).sum()
cross_elas_term_pt_time = (simulatedValues['Weighted prob. PT'] * simulatedValues['cross_elas_pt_time'] / denominator_pt).sum()
cross_elas_term_pt_cost = (simulatedValues['Weighted prob. PT'] * simulatedValues['cross_elas_pt_cost'] / denominator_pt).sum()

**Print the resulting aggregate direct elasticities**

In [26]:
print(f"Aggregate cross elasticity of car wrt PT time: {cross_elas_term_car_time:.3g}")
print(f"Aggregate cross elasticity of car wrt PT cost: {cross_elas_term_car_cost:.3g}")
print(f"Aggregate cross elasticity of PT wrt car time: {cross_elas_term_pt_time:.3g}")
print(f"Aggregate cross elasticity of PT wrt car cost: {cross_elas_term_pt_cost:.3g}")

Aggregate cross elasticity of car wrt PT time: 0.106
Aggregate cross elasticity of car wrt PT cost: 0.123
Aggregate cross elasticity of PT wrt car time: 0.0953
Aggregate cross elasticity of PT wrt car cost: 0.2


## Aggregate Arc Elasticities

**Define after scenario: Increase distance by 1km**

In [27]:
delta_dist = 1.
distance_km_scaled_after = (distance_km + delta_dist) / 5

**Define new utility function for the new scenario**

In [28]:
V_SM_after = ASC_SM + \
       BETA_DIST_MALE * distance_km_scaled_after * male + \
       BETA_DIST_FEMALE * distance_km_scaled_after * female + \
       BETA_DIST_UNREPORTED * distance_km_scaled_after * unreportedGender

**Define new utility functions dictionary**

In [29]:
V_after = {0: V_PT,
           1: V_CAR,
           2: V_SM_after}

**Define new model**

In [30]:
# prob_sm = models.nested(V,av,nests,2)
prob_sm_after = models.nested(V_after,av,nests,2)

**Calculate the arc elasticity**

In [31]:
direct_elas_sm_dist = (prob_sm_after - prob_sm) * distance_km / (prob_sm * delta_dist)

**Define what we want to simulate**

In [32]:
simulate = {'weight': normalized_Weight,
            'Prob. slow modes':prob_sm,
            'Prob. after slow modes':prob_sm_after,
            'direct_elas_sm_dist':direct_elas_sm_dist}

**Define the Biogeme object**

In [33]:
biogeme = bio.BIOGEME(database, simulate)
biogeme.modelName = "optima_nested_logit_elasticities_arc"

**Retrieve the names of the variables we want to use. Then retrieve the results from the model that we estimated earlier**

In [34]:
betas = biogeme.freeBetaNames

results = res.bioResults(pickleFile='optima_nested_logit.pickle')
betaValues = results.getBetaValues ()

**Perform the simulation**

In [35]:
simulatedValues = biogeme.simulate(betaValues)

**Calculate the elasticities**

In [36]:
simulatedValues['Weighted prob. slow modes'] = simulatedValues['weight'] * simulatedValues['Prob. slow modes']

denominator_sm = simulatedValues['Weighted prob. slow modes'].sum()

direct_elas_sm_dist = (simulatedValues['Weighted prob. slow modes'] * simulatedValues['direct_elas_sm_dist'] / denominator_sm).sum()

**Print the resulting aggregate direct elasticities**

In [37]:
print(f"Aggregate direct elasticity of slow modes wrt distance: {direct_elas_sm_dist:.3g}")

Aggregate direct elasticity of slow modes wrt distance: -1.01
